Copyright 2023 The TensorFlow Authors.


In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用 CNN 日报数据集的大语言模型的 TFX 流水线教程

在此 CodeLab 中，我们使用 KerasNLP 加载一个预训练的大语言模型 (LLM)（GPT-2 模型），并针对数据集进行微调。本演示中使用的数据集是 CNN 日报数据集。请注意，这里使用 GPT-2 只是为了演示端到端流程；本 CodeLab 中介绍的技术和工具可以转移到其他生成式语言模型，例如 Google T5。

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" https://tensorflow.google.cn/tfx/tutorials/tfx/CSV_Downloader_Component"> <img src="https://tensorflow.google.cn/images/tf_logo_32px.png">在 Tensorflow.org 上查看</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/tfx/tutorials/tfx/CSV_Downloader_Component.ipynb"> <img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 中运行</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/tfx/tutorials/tfx/CSV_Downloader_Component.ipynb"> <img width="32px" src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 GitHub 上查看源代码</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/tfx/tutorials/tfx/CSV_Downloader_Component.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a></td>
</table></div>

# 准备工作

Colab 提供了不同类型的运行时。请确保转到 **Runtime -&gt; Change runtime type** 并选择 GPU 硬件加速器运行时（应具有 &gt;12G 的系统 RAM 和约 15G 的 GPU RAM），因为您将对 GPT-2 模型进行微调。

# 安装

我们首先安装 TFX Python 软件包。

## 升级 Pip

为了避免在本地运行时升级系统中的 Pip，请检查以确保在 Colab 中运行。当然，可以对本地系统单独升级。

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

## 安装 TFX

目前，在 Colab 中与 Python 3.10 一起使用时，TFX 会发生问题。因此，简单地运行命令

```
!pip install -U tfx
```

来安装 tfx **会失败**。因此，请运行以下代码。

In [ ]:
%%shell
update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 3
curl -O https://bootstrap.pypa.io/get-pip.py
python get-pip.py

In [ ]:
# 1) TFX relies on an old version of google-api-core so we let google-auth float
# for the install. We grep it out below:
!grep -v google-auth /etc/requirements.core.in > requirements.txt

# 2) httplib2 should be included in /etc/requirements.core.in but it's not for
# reasons. We ensure it's included:
!grep httplib2 /etc/requirements.user.in >> requirements.txt

# 3) google.colab package is not available as a wheel. We symlink that in so
# it's on the sys.path of Python 3.8:
!mkdir /usr/local/lib/python3.8/dist-packages/google
!ln -s /usr/local/lib/python3.10/dist-packages/google/colab /usr/local/lib/python3.8/dist-packages/google/colab

# Now with those pre-requisites out of the way:
!pip install tfx==1.13.0 -r requirements.txt

In [ ]:
!pip install keras_nlp

# 导入

我们首先进行导入。

In [ ]:
from tensorflow import keras
from tfx.types import Channel
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

## 卸载 Shapely

TODO(b/263441833) 这是避免 ImportError 的临时解决方案。最终，应该通过支持最新版本的 Bigquery 来处理，而不是卸载其他额外的依赖项。


In [ ]:
!pip uninstall shapely -y

## 是否已重新启动运行时？

如果您使用的是 Google Colab，则在首次运行上方代码单元时必须重新启动运行时，方法是点击上方“RESTART RUNTIME”按钮或使用“Runtime &gt; Restart runtime…”菜单。这样做的原因是 Colab 加载软件包的方式。

检查 TensorFlow 和 TFX 版本。

我们来检查一下库版本。

In [ ]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

## 设置变量

有一些变量用于定义流水线。您可以根据需要自定义这些变量。默认情况下，流水线的所有输出都将在当前目录下生成。

# CSV 下载程序

为了使流水线更加高效并实现自动化，拥有一个接收要下载的 CSV 文件的下载链接的组件非常有用。此外，TFX 生产 ML 流水线的一个重要目标是收集包含有关流水线组件、其执行和所产生工件的信息的元数据。换句话说，元数据的目的是分析流水线组件的沿袭和调试问题，CSV 下载程序组件会帮助用户记录和跟踪有关数据源的信息，以及数据在进入流水线之前经历的预处理步骤的信息。在此部分中，我们声明一个名为 CSVdoc 的新工件，并开发一个自定义组件（CSV 下载程序），它存储有关数据集的信息并在 CSVdoc 工件的 URI 中下载 CSV 文件。

In [ ]:
from tfx.types import artifact
from tfx import types

Property = artifact.Property
PropertyType = artifact.PropertyType

URL_PROPERTY = Property(type=PropertyType.STRING)
PATH_PROPERTY = Property(type=PropertyType.STRING)

class CsvDoc(types.Artifact):
  """ Artifact that contains the CSV dataset.

     - 'url' : saves the source of the original data.
     - 'path': saves the path to the CSV file.
  """

  TYPE_NAME = 'CsvDoc'
  PROPERTIES = {
      'url' : URL_PROPERTY,
      'path': PATH_PROPERTY,
  }

In [ ]:
from absl import logging
import requests
import os
import tfx.v1 as tfx
from tfx.dsl.component.experimental.decorators import component

@tfx.dsl.components.component
def CsvDownloaderComponent(
    url: tfx.dsl.components.Parameter[str],
    file_name: tfx.dsl.components.Parameter[str],
    saved_file: tfx.dsl.components.OutputArtifact[CsvDoc],
) -> None:
  response = requests.get(url)
  saved_file.url = url
  if response.status_code == 200:
    file_path = os.path.join(saved_file.uri, file_name)
    saved_file.path = file_path
    url_content = response.content
    with open(file_path, 'wb') as csv_file:
      csv_file.write(url_content)
    logging.info(f"CSV file saved successfully at {file_path}")
  else:
    raise Exception("CSV file failed to be saved.")

In [ ]:
downloader = CsvDownloaderComponent(
  url = 'https://drive.google.com/uc?id=1YdZsJlRafqxiNSl0nHQkwR7rzrNlN9LI&export=download', file_name ='testing_doc.csv')

In [ ]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
context = InteractiveContext()

In [ ]:
context.run(downloader, enable_cache = False)